# RSI momentum trading strategy example

- This is a backtest example notebook
    - The notebook sees how the usage of trailing stop loss affects the results

# Set up

Set up Trading Strategy data client.


In [1]:
from tradeexecutor.utils.notebook import setup_charting_and_output
from tradingstrategy.client import Client

client = Client.create_jupyter_client()

# Render for Github web viewer
from tradeexecutor.utils.notebook import setup_charting_and_output, OutputMode
#setup_charting_and_output(OutputMode.static, image_format="png", width=1500, height=1000)
setup_charting_and_output()

Started Trading Strategy in Jupyter notebook environment, configuration is stored in /Users/moo/.tradingstrategy


# Load data

We use Binance data so we get a longer period of data.

In [2]:
import datetime
from tradingstrategy.timebucket import TimeBucket
from tradeexecutor.utils.binance import create_binance_universe

strategy_universe = create_binance_universe(
    ["BTCUSDT", "ETHUSDT"],   # Binance internal tickers later mapped to Trading strategy DEXPair metadata class
    candle_time_bucket=TimeBucket.d1,
    stop_loss_time_bucket=TimeBucket.h1,
    start_at=datetime.datetime(2019, 1, 1),  # Backtest for 5 years data
    end_at=datetime.datetime(2024, 2, 15),
    include_lending=False
)


  0%|          | 0/2 [00:00<?, ?it/s]

# Show loaded trading universe

Display generic troubleshooting information about the loaded data.

In [3]:
pairs = strategy_universe.data_universe.pairs  # Trading pairs metadata
candles = strategy_universe.data_universe.candles  # Candles for all trading pairs

print(f"Loaded {candles.get_candle_count():,} candles.")

for pair in pairs.iterate_pairs():
    pair_candles = candles.get_candles_by_pair(pair)
    first_close = pair_candles.iloc[0]["close"]
    first_close_at = pair_candles.index[0]
    print(f"Pair {pair} first close price {first_close} at {first_close_at}")

Loaded 3,654 candles.
Pair <Pair #1 BTC - USDT at exchange binance> first close price 3797.14 at 2019-01-01 00:00:00
Pair <Pair #2 ETH - USDT at exchange binance> first close price 139.1 at 2019-01-01 00:00:00


# Trading algorithm

In [4]:
from tradeexecutor.strategy.weighting import weight_equal, weight_by_1_slash_n, weight_passthrouh
from tradeexecutor.strategy.alpha_model import AlphaModel
import numpy as np
from tradeexecutor.strategy.pandas_trader.position_manager import PositionManager
from tradingstrategy.chain import ChainId
from typing import List, Dict

from pandas_ta.momentum import rsi
import pandas as pd

from tradeexecutor.strategy.trading_strategy_universe import TradingStrategyUniverse
from tradeexecutor.state.visualisation import PlotKind, PlotShape, PlotLabel
from tradeexecutor.state.trade import TradeExecution
from tradeexecutor.strategy.pricing_model import PricingModel
from tradeexecutor.state.state import State

# List of pair descriptions we used to look up pair metadata
our_pairs = [
    (ChainId.centralised_exchange, "binance", "BTC", "USDT"),
    (ChainId.centralised_exchange, "binance", "ETH", "USDT"),
]

rsi_days = 5  # The length of RSI indicator
eth_btc_rsi_days = 60  # The length of ETH/BTC RSI
rsi_high = 77  # RSI trigger threshold for decision making
rsi_low = 60  # RSI trigger threshold for decision making
allocation = 0.98 # Allocate 90% of cash to each position
lookback_candles = 120
minimum_rebalance_trade_threshold = 500.00  # Don't do trades that would have less than 500 USD value change
trailing_stop_loss=0.97


def decide_trades(
    timestamp: pd.Timestamp,
    strategy_universe: TradingStrategyUniverse,
    state: State,
    pricing_model: PricingModel,
    cycle_debug_data: Dict
) -> List[TradeExecution]:

    # Resolve our pair metadata for our two pair strategy
    position_manager = PositionManager(timestamp, strategy_universe, state, pricing_model)
    alpha_model = AlphaModel(timestamp)
    btc_pair = position_manager.get_trading_pair(our_pairs[0])
    eth_pair = position_manager.get_trading_pair(our_pairs[1])

    position_manager.log("decide_trades() start")

    #
    # Indicators
    #
    # Calculate indicators for each pair.
    #

    # Per-trading pair calcualted data
    close_prices = {btc_pair: None, eth_pair: None}  # Recent close prices
    current_rsi_values = {}  # RSI yesterday
    previous_rsi_values = {}  # RSI day before yesterday
    current_price = {}  # Close price yesterday
    momentum = {btc_pair: 0, eth_pair: 0}
    eth_btc_rsi_yesterday = None

    for pair in [btc_pair, eth_pair]:

        pair_candles = candles.get_last_entries_by_pair_and_timestamp(pair.internal_id, timestamp)
        close_prices[pair] = pair_candles["close"]

        assert pair_candles is not None
        rsi_series = rsi(pair_candles["close"], length=rsi_days)  # Will return None if the data buffer does not have enough days to look back

        # Reset indicators for this cycle and this pair
        current_rsi_values[pair] = None
        previous_rsi_values[pair] = None
        current_price[pair] = None
        price_today = None

        if len(pair_candles) > 0:
            # We have enough data to get the latest price
            current_price[pair] = price_today = pair_candles["close"][-1]

        if rsi_series is not None:
            current_val = rsi_series[-1]
            if np.isfinite(current_val):
                # We have enough data and good value for RSI
                assert 0 < current_val < 100, f"RSI sanity check failed: {pair}: {current_val}"  # Check we are in expected range
                current_rsi_values[pair] = current_val

            previous_val = rsi_series[-2]
            if np.isfinite(previous_val):
                # We have enough data and good value for RSI
                assert 0 < previous_val < 100, f"RSI sanity check failed: {pair}: {previous_val}"  # Check we are in expected range
                previous_rsi_values[pair] = previous_val

            previous_rsi_values[pair] = previous_val

    eth_btc_price = close_prices[eth_pair] / close_prices[btc_pair]
    eth_btc_rsi = rsi(eth_btc_price, length=eth_btc_rsi_days)
    if eth_btc_rsi is not None:
        eth_btc_rsi_yesterday = eth_btc_rsi[-1]
        momentum[eth_pair] = (eth_btc_rsi_yesterday - 50) ** 3
        momentum[btc_pair] = (50 - momentum[eth_pair]) ** 3

    #
    # Trading logic
    #

    for pair in [btc_pair, eth_pair]:

        existing_position = position_manager.get_current_position_for_pair(pair)
        pair_open = existing_position is not None
        pair_momentum = momentum.get(pair, 0)
        signal_strength = max(pair_momentum, 0.1)  # Singal strength must be positive, as we do long-only
        if pd.isna(signal_strength):
            signal_strength = 0
        alpha_model.set_signal(pair, 0)

        if pair_open:
            # We have existing open position for this pair,
            # keep it open by default unless we get a trigger condition below
            position_manager.log(f"Pair {pair} already open")
            alpha_model.set_signal(pair, signal_strength, trailing_stop_loss=trailing_stop_loss)

        if current_rsi_values[pair] and previous_rsi_values[pair]:

            # Check for RSI crossing our threshold values in this cycle, compared to the previous cycle
            rsi_cross_above = current_rsi_values[pair] >= rsi_high and previous_rsi_values[pair] < rsi_high
            rsi_cross_below = current_rsi_values[pair] < rsi_low and previous_rsi_values[pair] > rsi_low

            if not pair_open:
                # Check for opening a position if no position is open
                if rsi_cross_above:
                    position_manager.log(f"Pair {pair} crossed above")
                    alpha_model.set_signal(pair, signal_strength, trailing_stop_loss=trailing_stop_loss)
            else:
                # We have open position, check for the closure condition
                if rsi_cross_below:
                    position_manager.log(f"Pair {pair} crossed below")
                    alpha_model.set_signal(pair, 0)

    # Equally weight for all assets that
    alpha_model.select_top_signals(2)
    alpha_model.assign_weights(weight_passthrouh)
    alpha_model.normalise_weights()
    alpha_model.update_old_weights(state.portfolio)
    portfolio = position_manager.get_current_portfolio()
    portfolio_target_value = portfolio.get_total_equity() * allocation
    alpha_model.calculate_target_positions(position_manager, portfolio_target_value)
    trades = alpha_model.generate_rebalance_trades_and_triggers(
        position_manager,
        min_trade_threshold=minimum_rebalance_trade_threshold,
    )

    #
    # Visualisations
    #

    visualisation = state.visualisation  # Helper class to visualise strategy output

    visualisation.plot_indicator(
        timestamp,
        f"ETH",
        PlotKind.technical_indicator_detached,
        current_price[eth_pair],
        colour="blue",
    )

    # Draw BTC + ETH RSI between its trigger zones for this pair of we got a valid value for RSI for this pair

    # BTC RSI daily
    visualisation.plot_indicator(
        timestamp,
        f"RSI",
        PlotKind.technical_indicator_detached,
        current_rsi_values[btc_pair],
        colour="orange",
    )

    # ETH RSI daily
    visualisation.plot_indicator(
        timestamp,
        f"RSI ETH",
        PlotKind.technical_indicator_overlay_on_detached,
        current_rsi_values[eth_pair],
        colour="blue",
        label=PlotLabel.hidden,
        detached_overlay_name=f"RSI",
    )

    # Low (vertical line)
    visualisation.plot_indicator(
        timestamp,
        f"RSI low trigger",
        PlotKind.technical_indicator_overlay_on_detached,
        rsi_low,
        detached_overlay_name=f"RSI",
        plot_shape=PlotShape.horizontal_vertical,
        colour="red",
        label=PlotLabel.hidden,
    )

    # High (vertical line)
    visualisation.plot_indicator(
        timestamp,
        f"RSI high trigger",
        PlotKind.technical_indicator_overlay_on_detached,
        rsi_high,
        detached_overlay_name=f"RSI",
        plot_shape=PlotShape.horizontal_vertical,
        colour="red",
        label=PlotLabel.hidden,
    )

    if eth_btc_rsi_yesterday is not None:

        visualisation.plot_indicator(
            timestamp,
            f"ETH/BTC",
            PlotKind.technical_indicator_detached,
            eth_btc_price[-1],
            colour="grey",
        )

        if pd.notna(eth_btc_rsi_yesterday):
            visualisation.plot_indicator(
                timestamp,
                f"ETH/BTC RSI",
                PlotKind.technical_indicator_detached,
                eth_btc_rsi_yesterday,
                colour="grey",
            )

    state.visualisation.add_calculations(timestamp, alpha_model.to_dict())  # Record alpha model thinking

    return trades

/Users/moo/Library/Caches/pypoetry/virtualenvs/trade-executor-8Oz1GdY1-py3.10/lib/python3.10/site-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


# Backtest

In [5]:
import logging
from tradeexecutor.strategy.cycle import CycleDuration
from tradeexecutor.backtest.backtest_runner import run_backtest_inline

state, universe, debug_dump = run_backtest_inline(
    name="RSI multipair",
    engine_version="0.3",
    decide_trades=decide_trades,
    client=client,
    cycle_duration=CycleDuration.cycle_1d,
    universe=strategy_universe,
    initial_deposit=10_000,
    strategy_logging=False,
)

trade_count = len(list(state.portfolio.get_all_trades()))
print(f"Backtesting completed, backtested strategy made {trade_count} trades")

  0%|          | 0/157766400 [00:00<?, ?it/s]

Backtesting completed, backtested strategy made 239 trades


# Benchmark

Benchmark against

- Buy and hold BTC
- Buy and hold ETH

In [6]:
from tradeexecutor.visual.benchmark import visualise_benchmark

btc_pair = strategy_universe.data_universe.pairs.get_pair_by_human_description(our_pairs[0])
eth_pair = strategy_universe.data_universe.pairs.get_pair_by_human_description(our_pairs[1])

benchmark_indexes = pd.DataFrame({
    "BTC": strategy_universe.data_universe.candles.get_candles_by_pair(btc_pair)["close"],
    "ETH": strategy_universe.data_universe.candles.get_candles_by_pair(eth_pair)["close"],
})
benchmark_indexes["BTC"].attrs = {"colour": "orange", "name": "Buy and hold BTC"}
benchmark_indexes["ETH"].attrs = {"colour": "blue", "name": "Buy and hold ETH"}

fig = visualise_benchmark(
    name=state.name,
    portfolio_statistics=state.stats.portfolio,
    all_cash=state.portfolio.get_initial_deposit(),
    benchmark_indexes=benchmark_indexes,
    height=800
)

fig.show()

/var/folders/12/pbc59svn70q_9dfz1kjl3zww0000gn/T/ipykernel_63223/586607360.py:13: DeprecationWarning: This function is deprecated. Use visualise_equity_curve_benchmark instead
  fig = visualise_benchmark(


# Performance metrics

Compare popular portfolio performance metrics.

- Benchmark the strategy against buy and hold ETH portfolio

In [7]:
from tradeexecutor.visual.equity_curve import calculate_equity_curve, calculate_returns, generate_buy_and_hold_returns
from tradeexecutor.analysis.advanced_metrics import visualise_advanced_metrics, AdvancedMetricsMode

equity = calculate_equity_curve(state)
returns = calculate_returns(equity)
benchmark_returns = generate_buy_and_hold_returns(benchmark_indexes["ETH"])

metrics = visualise_advanced_metrics(
    returns,
    mode=AdvancedMetricsMode.full,
    benchmark=benchmark_returns,
)

display(metrics)

,Strategy,Buy and hold ETH
Start Period,2019-01-01,2019-01-01
End Period,2023-12-31,2023-12-31
Risk-Free Rate,0.0%,0.0%
Time in Market,21.0%,100.0%
Cumulative Return,182.13%,"1,540.45%"
CAGR﹪,23.05%,74.98%
Sharpe,0.98,1.09
Prob. Sharpe Ratio,99.34%,99.18%
Smart Sharpe,0.95,1.05
Sortino,2.13,1.59


# Trading metrics

Trading-related metrics like fees.



In [8]:
from tradeexecutor.analysis.trade_analyser import build_trade_analysis

analysis = build_trade_analysis(state.portfolio)
summary = analysis.calculate_summary_statistics()
display(summary.to_dataframe())

,0
Trading period length,1766 days 3 hours
Return %,182.13%
Annualised return %,37.64%
Cash at start,"$10,000.00"
Value at end,"$28,213.09"
Trade volume,"$3,931,324.46"
Position win percent,40.68%
Total positions,118
Won positions,48
Lost positions,70


# Stop-loss trigger data

- Examine how the strategy triggers stop losses


In [9]:
from tradeexecutor.analysis.stop_loss import analyse_stop_losses


df = analyse_stop_losses(state)
display(df)

,trading_pair,triggered,updates,duration,profit,initial,drift_up,drift_down,opening_stop_loss,lowest_stop_loss,highest_stop_loss,opening_price,closing_price,lowest_price,highest_price
position_id,,,,,,,,,,,,,,,
1,BTC-USDT,False,6,5 days 00:00:00,-1%,-3%,1%,0%,"$3,550.46","$3,550","$3,569","$3,662.10","$3,606.54","$3,660","$3,680"
2,ETH-USDT,True,6,2 days 01:00:00,1%,-3%,5%,0%,$115.74,$116,$121,$119.38,$120.86,$119,$125
3,ETH-USDT,True,11,1 days 03:00:00,9%,-3%,12%,0%,$129.76,$130,$145,$133.84,$144.83,$134,$150
4,BTC-USDT,True,13,5 days 15:00:00,-3%,-3%,7%,0%,"$3,780.43","$3,780","$4,048","$3,899.30","$3,788.09","$3,897","$4,173"
5,BTC-USDT,True,7,4 days 15:00:00,-2%,-3%,1%,0%,"$3,885.80","$3,886","$3,929","$4,007.98","$3,908.05","$4,006","$4,050"
6,BTC-USDT,True,14,1 days 05:00:00,17%,-3%,21%,0%,"$4,020.20","$4,020","$4,872","$4,146.61","$4,835.53","$4,145","$5,023"
7,BTC-USDT,True,6,0 days 11:00:00,-2%,-3%,1%,0%,"$5,326.18","$5,326","$5,399","$5,493.66","$5,377.17","$5,491","$5,566"
8,BTC-USDT,True,3,0 days 14:00:00,-2%,-3%,2%,0%,"$5,597.50","$5,598","$5,707","$5,773.51","$5,630.34","$5,771","$5,884"
9,BTC-USDT,True,44,4 days 12:00:00,22%,-3%,28%,0%,"$5,670.95","$5,671","$7,256","$5,849.26","$7,159.08","$5,846","$7,480"


Check the stop loss trigger update signal of a particular position.

In [12]:
from tradeexecutor.analysis.stop_loss import analyse_trigger_updates
from tradingstrategy.utils.format import format_percent, format_percent_2_decimals

p = state.portfolio.get_position_by_id(2)
position_data = {
    "Position id": p.position_id,
    "Duration": p.get_duration(),
    "Open price": p.get_opening_price(),
    "Close price": p.get_closing_price(),
    "Profit": format_percent_2_decimals(p.get_realised_profit_percent()),    
}

display(pd.DataFrame.from_dict(position_data, orient="index"))

df = analyse_trigger_updates(p)
display(df)

,0
Position id,2
Duration,"2 days, 1:00:00"
Open price,119.37966
Close price,120.85954
Profit,1%


,mid_price,stop_loss_before,stop_loss_after,take_profit_before,take_profit_after
timestamp,,,,,
2019-02-09 00:00:00,119.32,NaN,115.7404,None,None
2019-02-09 11:00:00,119.46,115.7404,115.8762,None,None
2019-02-09 14:00:00,119.53,115.8762,115.9441,None,None
2019-02-09 15:00:00,119.82,115.9441,116.2254,None,None
2019-02-10 23:00:00,121.61,116.2254,117.9617,None,None
2019-02-11 00:00:00,125.12,117.9617,121.3664,None,None
